In [ ]:
import os

import numpy as np

import shutil

from keras import callbacks

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedGroupKFold

from source.set_gpu import set_gpu
from source.load_mat_data import load_mat_data
from source.model.cnn import cnn
from source.save_result.save_history import save_history
from source.save_result.save_predict import save_predict

###################################################################################
### learning setting
DIR_PATH_LOAD_AND_Y_TUPLES = [
    ("/home/unixuser/cdproject/data/tmp/HC", 0),
    ("/home/unixuser/cdproject/data/tmp/ADHD", 1)
]
META_PATH_LOAD = "/home/unixuser/cdproject/data/public_data/raw/meta.txt"

N_SPLITS = 5

DIR_PATH_SAVE_RESULT = "/home/unixuser/cdproject/data/tmp"
os.makedirs(os.path.join(DIR_PATH_SAVE_RESULT), exist_ok=True)
###################################################################################

# gpu settiing
set_gpu()

# copy meta
shutil.copy(META_PATH_LOAD, DIR_PATH_SAVE_RESULT)
print(f"{META_PATH_LOAD} has been copied to {DIR_PATH_SAVE_RESULT}")

# initialization
x = None
y = None
sbj = None
num = None

#load mat data
for dir_path_load_arg, y_arg in DIR_PATH_LOAD_AND_Y_TUPLES:
     x_tmp, y_tmp, sbj_tmp, num_tmp = load_mat_data(dir_path_load_arg, y_arg)

     if x is None:
        x = x_tmp
        y = y_tmp
        sbj = sbj_tmp
        num = num_tmp
        continue
     
     x = np.concatenate((x, x_tmp), axis=0)
     y = np.concatenate((y, y_tmp), axis=0)
     sbj = np.concatenate((sbj, sbj_tmp), axis=0)
     num = np.concatenate((num, num_tmp), axis=0)

###################################################################################
###reshape x. 1d must be the number of data
x = np.transpose(np.expand_dims(x, axis=-1), (0, 2, 1, 3)) #x = nd array. 1d must be the number of data
###################################################################################
y = np.array(y) #y = 1d array. x의 label
sbj = np.array(sbj) #sbj = 1d array. x의 subject
num = np.array(num) #num = 1d array. x의 number

GKF = StratifiedKFold(n_splits=N_SPLITS)
SGKF = StratifiedGroupKFold(n_splits=N_SPLITS)
KFOLDS = [(GKF.split(x, y), 'StratifiedKFold'), (SGKF.split(x, y, sbj), 'StratifiedGroupKFold')]
for kfold, fold_name in KFOLDS:
    for fold_num, (train_index, test_index) in enumerate(kfold, start = 1):
        x_train = x[train_index]
        y_train = y[train_index]

        x_test = x[test_index]
        y_test = y[test_index]
        sbj_test = sbj[test_index]
        num_test = num[test_index]

        ###################################################################################
        ### get model / set model / train model / predict model
        model = cnn(x[0].shape)

        reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001, verbose=1)
        early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        checkpoint = callbacks.ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min')

        history = model.fit(x_train, y_train, epochs=50, verbose=0, validation_data=(x_test, y_test), callbacks=[reduce_lr, early_stopping, checkpoint])

        y_prob = model.predict(x_test)
        ###################################################################################

        # save result
        result_fold = f"{fold_name}_{fold_num}"
        print(f"--------------------{result_fold}--------------------")
        save_history(history, os.path.join(DIR_PATH_SAVE_RESULT, f"history_{result_fold}.mat"))
        save_predict(y_prob, y_test, sbj_test, num_test, os.path.join(DIR_PATH_SAVE_RESULT, f"predict_{result_fold}.mat"))